In [1]:
import os
print(os.getcwd())
# os.chdir("..")

from src.constants import *
from src.model.my_model import MyModel
from src.model.baselines import UNet, SWINUNETR
import torch
from src.utils.metrics import *
from torch.cuda.amp import GradScaler, autocast

/home/baris/Documents/AAAsegmentor/notebooks


In [2]:
device = "cuda"

In [3]:
s=128
example = torch.rand(size=(1, 1, 256, 256, 64)).to(device)
mask = torch.rand(size=(1, 3, s, s, s)).to(device)

example.dtype

torch.float32

In [4]:
model = MyModel(in_channels=1,
                mid_channels=4,
                out_channels=3,
                patch_size=(4,4,4),
                embed_dim=256,
                img_size=(256, 256, 64)).to(device)


In [5]:
with autocast():
    out = model(example)

Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)


In [6]:
out.shape

torch.Size([1, 3, 256, 256, 64])

In [6]:
import time
import os
import psutil
import gpustat
from contextlib import contextmanager

@contextmanager
def resource_monitor():
    # CPU and RAM usage monitoring
    process = psutil.Process(os.getpid())
    cpu_start = process.cpu_percent(interval=None)
    ram_start = process.memory_info().rss

    # GPU usage monitoring
    try:
        gpu_start = gpustat.new_query().jsonify()["gpus"][0]["memory.used"]
    except Exception as e:
        gpu_start = "Unable to retrieve GPU info: {}".format(e)

    # Time monitoring
    start_time = time.time()

    yield

    # Time monitoring
    elapsed_time = time.time() - start_time

    # CPU and RAM usage monitoring
    cpu_end = process.cpu_percent(interval=None)
    ram_end = process.memory_info().rss

    # GPU usage monitoring
    try:
        gpu_end = gpustat.new_query().jsonify()["gpus"][0]["memory.used"]
    except Exception as e:
        gpu_end = "Unable to retrieve GPU info: {}".format(e)

    print(f'Elapsed time: {elapsed_time} seconds')
    print(f'CPU used: {cpu_end - cpu_start} percent')
    print(f'RAM used: {ram_end - ram_start} bytes')

    if isinstance(gpu_start, float) and isinstance(gpu_end, float):
        print(f'GPU memory used: {gpu_end - gpu_start} MiB')
    else:
        print(f'GPU usage: {gpu_end}')


In [7]:

# Usage example:
with resource_monitor():
    out = model(example) 

Elapsed time: 1.1407158374786377 seconds
CPU used: 70.7 percent
RAM used: 1287770112 bytes
GPU usage: 21277


In [8]:
out.shape

torch.Size([4, 3, 256, 256, 64])

In [9]:
count_learnable_parameters(model.vit.embedders) / count_learnable_parameters(model.vit.vit)

0.01214992152190768

In [18]:
import torch
from flash_attn.flash_attention import FlashMHA

# Replace this with your correct GPU device
device = "cuda:0"

# Create attention layer. This is similar to torch.nn.MultiheadAttention,
# and it includes the input and output linear layers
flash_mha = FlashMHA(
    embed_dim=256, # total channels (= num_heads * head_dim)
    num_heads=16, # number of heads
    device=device,
    dtype=torch.float16,
)

# Run forward pass with dummy data
x = torch.randn(
    (8, 512, 256), # (batch, seqlen, embed_dim)
    device=device,
    dtype=torch.float16
)

output = flash_mha(x)[0]

In [15]:
out.shape

torch.Size([1, 3, 128, 128, 128])